In [22]:
import pandas_ta as ta
import time
from apscheduler.schedulers.blocking import BlockingScheduler
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
from backtesting import Strategy
from backtesting import Backtest
import yfinance as yf
from yahoo_fin import stock_info as si


In [23]:
def get_stock_candles_df(stock, startDate, endDate, interval):
    data = yf.download(stock, start=startDate, end = endDate, interval=interval)

    bb_bands = ta.bbands(data.Close, length=30, std=2)
    data = data.join(bb_bands)

    data['BB_width'] = (data['BBU_30_2.0'] - data['BBL_30_2.0'])

    data = data.reset_index()
    
    return data

In [24]:
def print_candles(df):    
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close']),
                    go.Scatter(x=df.index, y=df['BBU_30_2.0'],
                            line=dict(color='purple', width=1),
                            name="BBU_30_2.0"),
                go.Scatter(x=df.index, y=df['BBL_30_2.0'],
                            line=dict(color='purple', width=1),
                            name="BBL_30_2.0")
                                   
                ])
    
    buy = df.loc[df['near_reversal'] == 1].index
    sell = df.loc[df['near_reversal'] == -1].index

    for date in buy:
        fig.add_vline(x=date, line_width=1, line_dash="dash", line_color="green")
    for date in sell:
        fig.add_vline(x=date, line_width=1, line_dash="dash", line_color="red")

    fig.update_layout(width=1200, height=800)
    fig.show()

In [25]:
def find_near_reversal(df):
    df['near_reversal'] = 0
    for i in range(1, len(df)):
        top_10 = df.loc[i-1, 'BBU_30_2.0'] - (df.loc[i-1, 'BB_width'] * 0.1)
        bottom_10 = df.loc[i-1, 'BBL_30_2.0'] + (df.loc[i-1, 'BB_width'] * 0.1)

        if df.loc[i, 'Close'] > top_10:
            df.loc[i, 'near_reversal'] = -1
        elif df.loc[i, 'Close'] < bottom_10:
            df.loc[i, 'near_reversal'] = 1

    return df

In [26]:
def get_parameters(df):
    def SIGNAL():
        return df['total_signal']
    
    class MyStrat(Strategy):
        take_profit_coef = 1.0

        def init(self):
            super().init()
            self.signal1 = self.I(SIGNAL)

        def next(self):
            super().next()        
            stop_loss = self.data['SUPERT_12_3.0'][-1]
            
            if self.signal1 == 1:
                difference = self.data['Close'][-1] - stop_loss
                take_profit = self.data['Close'][-1] + (difference * self.take_profit_coef)
                if not self.position:
                    self.buy(sl=stop_loss, tp=take_profit, size= 0.25)
                elif self.position.is_short:
                    self.position.close()
                    self.buy(sl=stop_loss, tp=take_profit, size= 0.25)

            elif self.signal1 == -1:
                difference = stop_loss - self.data['Close'][-1]
                take_profit = self.data['Close'][-1] - (difference * self.take_profit_coef)
                if not self.position:
                    self.sell(sl=stop_loss, tp=take_profit, size= 0.25)
                elif self.position.is_long:
                    self.position.close()
                    self.sell(sl=stop_loss, tp=take_profit, size= 0.25)

    bt = Backtest(df, MyStrat, cash=10000, margin=1/30)
    stats, heatmap = bt.optimize(take_profit_coef=[i/10 for i in range(10, 26)],
                        maximize='Return [%]', max_tries=300,
                            random_state=0,
                            return_heatmap=True)
    print(stats)
    print(stats['_strategy'])
    print(stats['_trades'])

In [27]:
def get_all_us_stock_tickers():
    # nyse_tickers = si.tickers_nyse()
    nasdaq_tickers = si.tickers_nasdaq()
    # amex_tickers = si.tickers_amex()
    
    # Combine all tickers into one list and remove duplicates    
    return nasdaq_tickers

In [28]:
data = get_stock_candles_df('HRTS', '2022-01-01', '2024-12-31', '1d')
data = find_near_reversal(data)
print_candles(data)

[*********************100%%**********************]  1 of 1 completed
